TEST LOADER FOR PHOIBOS 150 DATA

Typical structure 
/2020/01 January/Day 10/Raw Data/8874

Withing the folder of the scan number there is an info.txt and a scan
In THE AVG FOLDER there is a set of images, averaged over every acquisition cycle
IN THE RAW FOLDER there are single acquisition cycles (difference: in EPFL we added the "save every n cycle function"
this RAW folder also contains a "LUT.txt" file

In [ ]:
#test the os package
import os

#get current directory
workingdir=os.getcwd()
#print(workingdir)
#navigate to example data
exampledatapath=os.path.realpath(os.path.join(workingdir, '..','tests\data\dataEPFL\\2020\\02 February\Day 17\Raw Data\9159'))
filelist=os.listdir(exampledatapath)
print("example data path= ",exampledatapath)
#this folders contains the AVG folder with the data, an info.txt vector and a scan vector
#filelist

#navigate to the image folder, for EPFL in the \AVG subfolder of the data (note that single scan cycles for partial load are in another subfolder - to be checked with Laurenz)
avgimagepath=os.path.realpath(os.path.join(workingdir, '..','tests\data\dataEPFL\\2020\\02 February\Day 17\Raw Data\9159\AVG'))
print("AVG images path= ",avgimagepath)
rawimagelist=os.listdir(avgimagepath)
#alternatively join the paths
#avgimagepath=os.path.join(exampledatapath,'AVG') 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fname=os.path.join(avgimagepath,rawimagelist[0]) 
numpy_array = np.loadtxt(fname, delimiter="\t")

h = plt.contourf(numpy_array)
plt.colorbar()
#plt.xlim(120,150)
#plt.ylim(150,200)

one of the first operations should be to filter the image to remove the grid via FFT filtering

In [ ]:
#load the module and import functions for reading the parameter table from the calib2d file
from importlib import reload
import specsanalyzer.convert
from specsanalyzer.convert import parameters_table
reload(specsanalyzer.convert)
from specsanalyzer.convert import GetParameters
reload(specsanalyzer.convert)



In [ ]:
#GET THE CALIBRATION FILE NAME FROM THE EXAMPLE FOLDER
calib2dfilename=os.path.realpath(os.path.join(workingdir, '..','tests\data\dataEPFL\phoibosEPFL.txt'))
#GET THE INFO FILE NAME FROM THE EXAMPLE FOLDER, SCAN 9159
#navigate to example data
exampledatapath=os.path.realpath(os.path.join(workingdir, '..','tests\data\dataEPFL\\2020\\02 February\Day 17\Raw Data\9159'))
filelist=os.listdir(exampledatapath)
str_match = list(filter(lambda x: 'info.txt' in x, filelist))
infofilename=os.path.join(exampledatapath,str_match[0]) 

calibrationmatrixfull=parameters_table(calib2dfilename)


In [ ]:
calibrationmatrixinterp=GetParameters(infofilename,calib2dfilename)

In [ ]:
#let's make and test the  polynomial fit
ek=10
ep=20
eshift=np.array([-0.05,0,0.05])
currentdamatrix=calibrationmatrixinterp
print(type(eshift))
da_energy=eshift*ep+ek*np.ones(eshift.shape)

def calculate_polynomial_Coef_Da(ek,ep,eshift,currentdamatrix):
    
    #get the Das from the damatrix
    #da1=currentdamatrix[0][:]
    #da3=currentdamatrix[1][:]
    #da5=currentdamatrix[2][:]
    #da7=currentdamatrix[3][:]

    #calcualte the energy values for each da, given the eshift
    da_energy=eshift*ep+ek*np.ones(eshift.shape)

    #create the polinomial coeffiecient matrix, each is a third order polinomial 
    
    dapolymatrix=np.zeros(currentdamatrix.shape)

    for i in range(0,currentdamatrix.shape[0]):
        #igor uses the fit poly 3, which should be a parabola
        dapolymatrix[i][:]=np.polyfit(da_energy, currentdamatrix[i][:], 2).transpose()
    return dapolymatrix

dapolymatrix=calculate_polynomial_Coef_Da(ek,ep,eshift,currentdamatrix)

In [ ]:
polcoef = np.polyfit(da_energy, currentdamatrix[0][:], 2)

plt.plot(da_energy,currentdamatrix[0][:])

energy_full=np.linspace(ek-ep*0.1,ek+ep*0.1,128)
polval=np.polyval(polcoef,energy_full)
polval2=np.polyval(dapolymatrix[0][:],energy_full)
plt.plot(energy_full,polval)
plt.plot(energy_full,polval2)

In [ ]:
from specsanalyzer.convert import calculate_polynomial_coef_da
reload(specsanalyzer.convert)

In [ ]:
dapolymatrix2=calculate_polynomial_coef_da(ek,ep,eshift,currentdamatrix)
dapolymatrix-dapolymatrix2

In [ ]:
#let's now calculate the mcp_position_mm and zinner functions

def zinner(ek,angle,dapolymatrix):
    #poly(D1, Ek )*(Ang) + 10^-2*poly(D3, Ek )*(Ang)^3 + 10^-4*poly(D5, Ek )*(Ang)^5 + 10^-6*poly(D7, Ek )*(Ang)^7
    result=0
    for i in range(0,dapolymatrix.shape[0]):
        #igor uses the fit poly 3, which should be a parabola
        result=result+ 10**(-(2*i))*angle**(1+2*i)*np.polyval(dapolymatrix[i][:],ek)
        
    return result

def zinner_diff(ek,angle,dapolymatrix):
    #poly(D1, Ek ) + 3*10^-2*poly(D3, Ek )*(Ang)^2 + 5*10^-4*poly(D5, Ek )*(Ang)^4 + 7*10^-6*poly(D7, Ek )*(Ang)^6
    result=0
    for i in range(0,dapolymatrix.shape[0]):
        #igor uses the fit poly 3, which should be a parabola
        result=result+ (2*i+1)*10**(-(2*i))*angle**(2*i)*np.polyval(dapolymatrix[i][:],ek)
        
    return result


def mcp_position_mm(ek,angle,ainner,dapolymatrix):
    
    mask=np.greater_equal(np.abs(angle),ainner)
   
    result=np.zeros(angle.shape)#ideally has to be evaluated on a mesh

    result = np.where(mask, zinner(ek,angle,dapolymatrix), np.sign(angle)*zinner(ek,angle,dapolymatrix)+(abs(angle)-ainner)*zinner_diff(ek,angle,dapolymatrix))
   
    return result
#if np.greater_equal(np.abs(angle),ainner) : #values larger than the ainner have a different treatment
#    result= zinner(ek,angle,dapolymatrix)
#else:
#    result= np.sign(angle)*zinner(ek,angle,dapolymatrix) + (abs(angle)-ainner)*zinner_diff(ek,angle,dapolymatrix)

In [ ]:

#mcp_position_mm(1,1,15,dapolymatrix)

angles=np.linspace(-18,18,1024)
energies=np.linspace(10,20,2048)

ainner=15

anglemesh,energymesh=np.meshgrid(angles,energies)
testimage=mcp_position_mm(energymesh,anglemesh,15,dapolymatrix)



In [ ]:
reload(specsanalyzer.convert)
from specsanalyzer.convert import mcp_position_mm

In [ ]:
angles=np.linspace(-18,18,1024)
energies=np.linspace(10,20,2048)

ainner=15

anglemesh,energymesh=np.meshgrid(angles,energies)

testimage=mcp_position_mm(energymesh,anglemesh,15,dapolymatrix)

In [ ]:

plt.contourf(angles,energies,testimage)
plt.colorbar()

// calculates the actual correction matrix
function Calculate_MatrixCorrection()

	SetDataFolder root:Data:tmpData:'Analyser':

	NVAR PixelSize, magnification, E_Offset_px, Ang_Offset_px, LensMode, Ek, Ep, De1, aInner
	NVAR EkinLow, EkinHigh, AzimuthLow, AzimuthHigh
	NVAR Binning
	NVAR Edge_pos, Edge_Slope
	
	String Str
	// for full frames, if cropped in program we need to do something else?
	Variable nx_pixel=1376/Binning, ny_pixel=1040/Binning
	Make/O/N=(nx_pixel, ny_pixel) MCP_Position_mm_Matrix=NaN, Angular_Correction=NaN
	Make/O/N=(nx_pixel) w_E, E_Correction=NaN
	Make/O/N=(ny_pixel) w_Ang

	Variable Edge_Coef
	
	// Just linear scaling according to the ranges from the calibration file. What limits are they done for?
	//w_E[] = EkinLow+p/(nx_pixel-1)*(EkinHigh-EkinLow)
	//w_Ang[] = AzimuthLow+p/(ny_pixel-1)*(AzimuthHigh-AzimuthLow)
	// Allow 20% more range
	//w_Ang[] = AzimuthLow*1.2+(p-Ang_Offset_px)/(ny_pixel-1)*(AzimuthHigh-AzimuthLow)*1.2
	// store positions in wave scaling of AngularCorrection now!
	setscale/P x, EkinLow, (EkinHigh-EkinLow)/nx_pixel, "eV", MCP_Position_mm_Matrix, Angular_Correction, E_Correction
	setscale/P y, AzimuthLow*1.2, (AzimuthHigh-AzimuthLow)*1.2/ny_pixel, "deg", MCP_Position_mm_Matrix, Angular_Correction

	// energy correction? What does it do? Seems to be strictly linear...
	// back-calculation of the pixel position from the energy in the w_E wave. Used for the proper interpolation later
	//E_Correction = limit( round( (w_E- Ek)/Ep/de1 / magnification/(PixelSize*Binning) + nx_pixel/2 + E_Offset_px)  , 0, nx_pixel)
	// no rounding for proper interpolation
	//E_Correction = (w_E- Ek)/Ep/de1 / magnification/(PixelSize*Binning) + nx_pixel/2 + E_Offset_px
	E_Correction = (x- Ek)/Ep/de1 / magnification/(PixelSize*Binning) + nx_pixel/2 + E_Offset_px
	// this is a Matrix with the mm-position for the energies and angles. This uses all the calibration parameters.
	MCP_Position_mm_Matrix = MCP_Position_mm( x, y )
	// Matrix that stores pixel position in the angular row for all energies and angles
	//Angular_Correction[][] = limit( round( MCP_Position_mm_Matrix(p)(q)/magnification/(PixelSize*Binning) + ny_pixel/2 + Ang_Offset_px)  , 0, ny_pixel)
	// do proper interpolation later
	Angular_Correction[][] = MCP_Position_mm_Matrix[p][q]/magnification/(PixelSize*Binning) + ny_pixel/2 + Ang_Offset_px
	
	Str="LensMode:;Ek:;PE:;Binning:"
	Str=ReplaceNumberByKey("LensMode", Str, LensMode)
	Str=ReplaceNumberByKey("Ek", Str, Ek)
	Str=ReplaceNumberByKey("PE", Str, Ep)
	Str=ReplaceNumberByKey("Binning", Str, binning)
	Note/K Angular_Correction, Str
	
	string temp = note(Angular_correction)
	// calculate the Jacobian determinant for Normalization
	duplicate/o Angular_Correction, Jacobian_Determinant, w_dxdE, w_dxdA, w_dydE, w_dydA
	// Energy derivative of angular correction
	differentiate/DIM=0 w_dydE
	// angular derivative of angular correction
	differentiate/DIM=1 w_dydA
	// energy derivative of energy correction
	duplicate/o E_correction, w_temp
	differentiate w_temp
	w_dxdE = w_temp[p]
	// angular derivative of energy correction is flat
	w_dxdA = 0
	Jacobian_Determinant = abs(w_dxdE*w_dydA - w_dydE*w_dxdA)
	killWaves/z w_dxdE, w_dxdA, w_dydE, w_dydA, w_temp
	
end

In [ ]:

""" NVAR PixelSize, magnification, E_Offset_px, Ang_Offset_px, LensMode, Ek, Ep, De1, aInner
	NVAR EkinLow, EkinHigh, AzimuthLow, AzimuthHigh
	NVAR Binning
	NVAR Edge_pos, Edge_Slope
	
	String Str
	// for full frames, if cropped in program we need to do something else?
	Variable nx_pixel=1376/Binning, ny_pixel=1040/Binning
	Make/O/N=(nx_pixel, ny_pixel) MCP_Position_mm_Matrix=NaN, Angular_Correction=NaN
	Make/O/N=(nx_pixel) w_E, E_Correction=NaN
	Make/O/N=(ny_pixel) w_Ang

	Variable Edge_Coef """


def calculate_matrixcorrection(ek,angle,dapolymatrix,de1,erange,arange,ainner,nx_pixel,ny_pixel,pixelsize,binning,magnification):
	res=1
	return res

In [ ]:
""" //~~~~~~~~~~~~~~~~~All the parameters needed to do the angular correction
//	LensMode			from user
//	Ek					from user
//	Ep					from user
//	E_Offset_px			from user
//	Ang_Offset_px		from user
//	Binning				from user
//	Edge_pos			from user
//	Edge_Slope			from user
//	eShift				from phoibos100.Calib2D
//	eRange				from phoibos100.Calib2D
//	aRange				from phoibos100.Calib2D
//	De1					from phoibos100.Calib2D
//	aInner				from phoibos100.Calib2D
//	Da3 - Da7			from phoibos100.Calib2D
//	WF					4.2eV or from CCDAcquire
//	PixelSize			0.00645 or from CCDAcquire
//	magnification			4.54 or from CCDAcquire
//~~~~~~~~~~~~~~~~~All the parameters needed to do the angular correction """


specsdictionary = {
  "lensmode": "WAM",
  "ek": 12,
  "ep": 20,
  "E_Offset_px" : 12
}
specsdictionary["lensmode"]



#here's a good way to create a dictionary from the info txt
def get_pair(line):
    key, sep, value = line.strip().partition(":")
    return key, value

with open(infofilename) as fd:    
    d = dict(get_pair(line) for line in fd)

d["ScanType"]








In [ ]:
filelist

In [ ]:
#the package regex seems to be the one for analysing the calib2d
#check this out
#  https://stackoverflow.com/questions/4146009/python-get-list-indexes-using-regular-expression


import re